In [ ]:
data_folder = "/mnt/data/human/covid2024"
metainfo_file = "/mnt/data/human/covid2024/metadata.tsv"
sequences_file = "/mnt/data/human/covid2024/sequences.fasta"
low_sequences_file = "/mnt/data/human/covid2024/low_sequences.fasta"
hi_sequences_file = "/mnt/data/human/covid2024/hi_sequences.fasta"

In [ ]:
with open(metainfo_file) as fh:
    for line in fh:
        d = line.strip().split("\t")
        print(list(enumerate(d)))
        break

In [ ]:
metainfo_file = "/mnt/data/human/covid2024/metadata.tsv"

name2meta = {}
name2date = {}
name2aa_muts = {}
name2acc = {}
acc2name = {}
with open(metainfo_file) as fh:
    for i, line in enumerate(fh):
        if i % 100000 == 0:
            print(i, end=" ")
        if i > 0:
            d = line.strip().split("\t")
            name2meta[d[0]] = d
            name2date[d[0]] = d[5]
            name2aa_muts[d[0]] = d[16].split(",")
            name2acc[d[0]] = d[4]
            acc2name[d[4]] = d[0]

In [ ]:
complete_hicov_ids = set([x for x in name2meta 
                              if name2meta[x][19] == "True" and name2meta[x][20] == "True"
                         ])

In [ ]:
# separate low and hi genomes

from trseeker.seqio.fasta_file import sc_iter_fasta_brute

with open(low_sequences_file, "w") as fw_low:
    with open(hi_sequences_file, "w") as fw_hi:
        for i, (header, seq) in enumerate(sc_iter_fasta_brute(sequences_file)):
            if i % 100000 == 0:
                print(i, end=" ")
            real_header = header[1:].split("|")[0]
            if real_header in complete_hicov_ids:
                fw_hi.write(f"{header}\n{seq}\n")
            else:
                fw_low.write(f"{header}\n{seq}\n")
        

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#
#@created: 08.08.2023
#@author: Aleksey Komissarov
#@contact: ad3002@gmail.com

from collections import defaultdict
import os
import argparse
from trseeker.seqio.fasta_file import sc_iter_fasta_brute


if __name__ == '__main__':

    parser = argparse.ArgumentParser(description='Fasta to variants for covid.')
    parser.add_argument('-f','--fasta', help='Fasta file', required=True)
    parser.add_argument('-i','--start', help='What million of reads process', required=True)
    args = vars(parser.parse_args())
    start = int(args["start"])
    sequences_file = args["fasta"]

    temp_fa = f"temp.low.{start}.fa" 
    temp_minimap2 = f"temp.low.{start}.minimap2"

    with open(temp_minimap2, "w") as fh:
        pass

    for iid, (header, sequence) in enumerate(sc_iter_fasta_brute(sequences_file)):
        if iid < start * 100000:
            continue
        if iid > (start + 1) * 100000:
            break
        if iid and iid % 10000 == 0:
            print(f"{start}:{iid}")
        name = header[1:].split("|")[0]
        with open(temp_fa, "w") as fh:
            fh.write(">%s\n%s" % (name, sequence))
        
        command = "minimap2 -c --cs /home/akomissarov/Dropbox/workspace/scrappers/gisaid/ref.fa %s 2> /dev/null | sort -k6,6 -k8,8n | k8 /home/skliver/Soft/minimap2/misc/paftools.js call -L20000 -f /home/akomissarov/Dropbox/workspace/scrappers/gisaid/ref.fa - 2> /dev/null >> %s" % (temp_fa, temp_minimap2)
        os.system(command)


In [ ]:
list(complete_hicov_ids)[:10]

In [ ]:
### Load all data and save processed

from PyExp import sc_iter_filepath_folder
from trseeker.seqio.fasta_file import sc_iter_fasta_brute

import os
from collections import defaultdict


id2name = {}
id2date = {}
vid = 0
for iid, (header, sequence) in enumerate(sc_iter_fasta_brute(hi_sequences_file)):
    if iid % 1000000 == 0:
        print(iid, end=" ")
    name, collection_date = header[1:].split("|")[:2]
    id2name[vid] = name
    id2date[vid] = collection_date
    vid += 1

In [ ]:
id2name = id2names
len(id2name), len(id2date)

In [ ]:
with open(id2names_file, "w") as fh:
    for vid in id2date:
        fh.write("%s\t%s\t%s\n" % (vid, id2name[vid], id2date[vid]))

In [ ]:
hi_folder_pattern = "/mnt/data/human/covid2024/temp_hi/temp.%s.minimap2"

### My fuckup: I forgot to save virus name in vcf stacks, I how to fix it and remove rereading a huge fasta file

mutation2names = defaultdict(list)
name2mutations = defaultdict(list)
mutations = set()


vid = -1
for i in range(160):
    minimap2_file_name = hi_folder_pattern % i
    print(f"{i}/160", end=" ")
    with open(minimap2_file_name) as fh:
        for line in fh:
            if line.startswith("##fileformat=VCFv4.1"):
                vid += 1
            if line.startswith("#"):
                continue
            d = line.strip().split("\t")
            start = int(d[1])
            ref = d[3]
            alt = d[4]
            name = id2name[vid]
            mutation = "%s_%s_%s" % (start, ref, alt)
            mutation2names[mutation].append(name)
            name2mutations[name].append(mutation)
            mutations.add(mutation)


In [ ]:
mutations = list(mutations)
mutations.sort(key=lambda x: int(x.split("_")[0]))

In [ ]:
len(name2mutations)

In [ ]:
len(mutations)

In [ ]:
file_name = "/home/akomissarov/Dropbox/mutations.hi.tsv"
with open(file_name, "w") as fw:
    for name in mutations:
        fw.write(f"{name}\n")

file_name = "/home/akomissarov/Dropbox/mutation2names.hi.tsv"
with open(file_name, "w") as fw:
    for mutation in mutation2names:
        loc_muts = ",".join(map(str,mutation2names[mutation]))
        fw.write(f"{mutation}\t{len(mutation2names[mutation])}\t{loc_muts}\n")
          
file_name = "/home/akomissarov/Dropbox/name2mutations.hi.tsv"
with open(file_name, "w") as fw:
    for name in name2mutations:
        muts = ",".join(map(str,name2mutations[name]))
        fw.write(f"{name}\t{len(name2mutations[name])}\t{muts}\n")


In [ ]:
# id2names_file = "/mnt/data/human/covid2024/id2names.txt" # id name date
# with open(id2names_file, "w") as fh:
#     for vid in id2names:
#         fh.write("%s\t%s\t%s\n" % (vid, id2name[vid], id2date[vid]))

In [ ]:
!head /mnt/data/human/covid2024/id2names.txt

In [ ]:
### mut2mut
mut2mut = defaultdict(int)
mut2mut_by_month = {}
for j, name in enumerate(name2mutations):
    if j % 100000 == 0:
        print(j, end=" ")
    month = name2date[name][:7] # 2021-07-24
    mut2mut_by_month.setdefault(month, defaultdict(int))
    muts = name2mutations[name]
    for i, mut1 in enumerate(name2mutations[name]):
        if len(mutation2names[mut1]) < 2:
            continue
        for mut2 in name2mutations[name][i+1:]:
            if len(mutation2names[mut2]) < 2:
                continue
            mut2mut[(mut1, mut2)] += 1
            mut2mut_by_month[month][(mut1, mut2)] += 1

In [ ]:
file_name = "/home/akomissarov/Dropbox/mut2mut_by_month.tsv"

with open(file_name, "w") as fw:
    for month in mut2mut_by_month:
        for (mut1, mut2), val in mut2mut_by_month[month].items():
            fw.write(f"{month}\t{mut1}\t{mut2}\t{val}\n")

In [ ]:
id2names = {}
id2date = {}
name2date = {}

vid = 1
for iid, (header, sequence) in enumerate(sc_iter_fasta_brute(hi_sequences_file)):
    if iid % 1000000 == 0:
        print(iid, end=" ")
    name, collection_date = header[1:].split("|")[:2]
    id2names[vid] = name
    id2date[vid] = collection_date
    name2date[name] = collection_date
    vid += 1

In [ ]:
id2date

In [ ]:
len(mutation2names)

In [ ]:
### data readers

file_name = "/home/akomissarov/Dropbox/mutations.hi.tsv"
mutations = []
with open(file_name) as fh:
    for line in fh:
        mut = line.strip()
        mutations.append(mut)

In [ ]:
mutation2x = {}
for i, mut in enumerate(mutations):
    mutation2x[mut] = i + 1

In [ ]:
mutation2x_coord = {}
for i, mut in enumerate(mutations):
    mutation2x_coord[mut] = int(mut.split("_")[0])

In [ ]:
len(mut2mut)

In [ ]:
### transform to bins:
mutation2x_coord = {}
mut2mut1bp = defaultdict(int)
for i,(x,y) in enumerate(mut2mut):
    if i and i % 1000000 == 0:
        print(i, end=" ")
    val = mut2mut[(x,y)]
    x = int(x.split("_")[0])
    y = int(y.split("_")[0])
    mut2mut1bp[(x,y)] += val
    mut2mut1bp[(y,x)] += val


In [ ]:
### transform to bins:
mutation2x_coord = {}
mut2mut100bp = defaultdict(int)
for i,(x,y) in enumerate(mut2mut):
    if i and i % 1000000 == 0:
        print(i, end=" ")
    val = mut2mut[(x,y)]
    x = int(x.split("_")[0]) // 100
    y = int(y.split("_")[0]) // 100
    mut2mut100bp[(x,y)] += val
    mut2mut100bp[(y,x)] += val


In [ ]:
### transform to bins:
mutation2x_coord = {}
mut2mut10bp = defaultdict(int)
for i,(x,y) in enumerate(mut2mut):
    if i and i % 1000000 == 0:
        print(i, end=" ")
    val = mut2mut[(x,y)]
    x = int(x.split("_")[0]) // 10
    y = int(y.split("_")[0]) // 10
    mut2mut10bp[(x,y)] += val
    mut2mut10bp[(y,x)] += val


In [ ]:
# file_mut2mut = "/home/akomissarov/Dropbox/mut2mut.tsv"

# with open(file_mut2mut, "w") as fw:
#     for (mut1, mut2) in mut2mut:
#         fw.write(f"{mut1}\t{mut2}\t{mutation2x[mut1]}\t{mutation2x[mut2]}\t{mut2mut[(mut1, mut2)]}\n")

In [ ]:
max_value = max(mut2mut10bp.values())
max_value, len(mut2mut10bp)

In [ ]:
max_value = max(mut2mut100bp.values())
max_value, len(mut2mut100bp)

In [ ]:
max_value = max(mut2mut1bp.values())
max_value, len(mut2mut1bp)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

data_dict = {(x, y): mut2mut1bp[(x,y)]/max_value for x,y in mut2mut1bp if mut2mut1bp[(x,y)]/max_value > 0.001}
x = [coord[0] for coord in data_dict.keys()]
y = [coord[1] for coord in data_dict.keys()]
intensity = list(data_dict.values())
plt.figure(figsize=(30, 30))
plt.scatter(x, y, c=intensity, cmap='viridis', s=1)  # s=1 makes the marker size small
plt.xlabel("X")
plt.ylabel("Y")
plt.title("Dot Plot with Intensity")
plt.savefig("filename.png")
plt.close()